In [161]:
#mettre ici les bibliotheques dont on a besoin
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, StackingRegressor
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet


## Sommaire
1. Import
2. Préparation
3. Baseline
4. Modèles linéaires
5. Modèles d’arbres
6. Méthodes d’ensemble
7. Conclusion

CSV Nettoyé

In [162]:
print("\n1. CHARGEMENT ET PRÉPARATION DES DONNÉES")
df_work = pd.read_csv("clean_seattle.csv")
print(f"Shape des données : {df_work.shape}")
df_work.head()



1. CHARGEMENT ET PRÉPARATION DES DONNÉES
Shape des données : (689, 48)


,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,DefaultData,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity,log_TotalGHGEmissions,log_SiteEnergyUse_kBtu,BuildingAge,AvgFloorArea,GFATotal_per_Building
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,659000030,...,False,Compliant,Unknown,249.98,2.83,5.525373,15.793246,89,7369.500000,88434.0
1,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,659000475,...,False,Compliant,Unknown,2089.28,2.19,7.645053,18.100297,47,23319.756098,956110.0
2,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,659000640,...,False,Compliant,Unknown,286.43,4.67,5.660979,15.731637,90,6132.000000,61320.0
3,9,2016,Nonresidential COS,Other,West Precinct,810 Virginia St,Seattle,WA,98101.0,660000560,...,False,Compliant,Unknown,301.81,3.10,5.713106,16.307609,17,48644.000000,97288.0
4,10,2016,NonResidential,Hotel,Camlin,1619 9th Avenue,Seattle,WA,98101.0,660000825,...,False,Compliant,Unknown,176.14,2.12,5.176940,15.566239,90,7546.181818,83008.0


Variables

In [163]:
num_cols = [
    'PropertyGFATotal', 'AvgFloorArea',
    'NumberofFloors', 'NumberofBuildings', 
    'BuildingAge', 'GFATotal_per_Building',
    'Latitude', 'Longitude'
]
#  pour le moment on ne prend pas 'ENERGYSTARScore'
cat_cols = ['PrimaryPropertyType']
target = 'log_TotalGHGEmissions'
print(f"✓ Variables numériques : {len(num_cols)}")
print(f"✓ Variables catégorielles : {len(cat_cols)}")
print(f"✓ Variable cible : {target}")

# Préprocesseur
preproc = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols)
])

✓ Variables numériques : 8
✓ Variables catégorielles : 1
✓ Variable cible : log_TotalGHGEmissions


Mise en place de la validation croisée

In [164]:
# configuration de la validation croisée
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Liste des métriques 
scoring = {
    'MAE':  'neg_mean_absolute_error',
    'RMSE': 'neg_root_mean_squared_error', 
    'R2':   'r2'
}

In [180]:
print('Je choisis ici de prendre la métrique RMSE car elle pénalise fortement les sous ou sur-estimations ce qui est crucial dans les données de type émission de carbone,\nelle est exprimée dans la même unité que la cible\net enfin elle complète la mesure du R² en donnant une mesure concrète de l erreur moyenne attendue ')

Je choisis ici de prendre la métrique RMSE car elle pénalise fortement les sous ou sur-estimations ce qui est crucial dans les données de type émission de carbone,
elle est exprimée dans la même unité que la cible
et enfin elle complète la mesure du R² en donnant une mesure concrète de l erreur moyenne attendue 


Préparation des données

In [165]:

# Sélection X, y
X = df_work[num_cols + cat_cols].copy()
y = df_work[target]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print(f"✓ Split réalisé : {len(X_train)} train, {len(X_test)} test")

✓ Split réalisé : 551 train, 138 test


In [166]:

print("1. MODÈLE BASELINE (DummyRegressor)")
print("-"*50)

dummy_pipe = Pipeline([
    ('prep', preproc),
    ('model', DummyRegressor(strategy='mean'))
])

# Cross-validation du baseline
cv_dummy = cross_validate(dummy_pipe, X_train, y_train, cv=cv, scoring=scoring)

baseline_r2 = cv_dummy['test_R2'].mean()
baseline_rmse = -cv_dummy['test_RMSE'].mean()
baseline_mae = -cv_dummy['test_MAE'].mean()

print(f"Baseline (DummyRegressor - moyenne) :")
print(f"  R² CV    : {baseline_r2:.4f} ± {cv_dummy['test_R2'].std():.4f}")
print(f"  RMSE CV  : {baseline_rmse:.4f} ± {cv_dummy['test_RMSE'].std():.4f}")
print(f"  MAE CV   : {baseline_mae:.4f} ± {cv_dummy['test_MAE'].std():.4f}")

1. MODÈLE BASELINE (DummyRegressor)
--------------------------------------------------


Baseline (DummyRegressor - moyenne) :
  R² CV    : -0.0066 ± 0.0062
  RMSE CV  : 1.4870 ± 0.1222
  MAE CV   : 1.1643 ± 0.1213


In [167]:
print("\n2. RÉGRESSION LINÉAIRE SIMPLE")
print("-" * 50)

# La régression linéaire sert de baseline pour comprendre
# les relations linéaires dans nos données avant d'ajouter de la régularisation
linear_pipe = Pipeline([('prep', preproc), ('model', LinearRegression())])
cv_linear = cross_validate(linear_pipe, X_train, y_train, cv=cv, scoring=scoring)

print(f"Régression Linéaire :")
print(f"  R² CV    : {cv_linear['test_R2'].mean():.4f} ± {cv_linear['test_R2'].std():.4f}")
print(f"  RMSE CV  : {-cv_linear['test_RMSE'].mean():.4f} ± {cv_linear['test_RMSE'].std():.4f}")
print(f"  MAE CV   : {-cv_linear['test_MAE'].mean():.4f} ± {cv_linear['test_MAE'].std():.4f}")


2. RÉGRESSION LINÉAIRE SIMPLE
--------------------------------------------------
Régression Linéaire :
  R² CV    : 0.2266 ± 0.2691
  RMSE CV  : 1.2752 ± 0.1674
  MAE CV   : 0.9305 ± 0.0767


In [168]:

print("3. MODÈLES RÉGULARISÉS (Ridge, Lasso, ElasticNet)")
print("-"*50)

#  La régularisation aide à :
# - Ridge : réduire l'overfitting en pénalisant les coefficients élevés
# - Lasso : sélection de variables en annulant certains coefficients
# - ElasticNet : combinaison des avantages de Ridge et Lasso
# modèles linéaires

# Pipeline de base
reg_pipe = Pipeline([
    ('prep', preproc),
    ('model', Ridge())  # Placeholder qui sera remplacé par GridSearchCV
])

# Grille de paramètres - pour tester plusieurs combinaisons d'hyperparamètres pour donner le meilleur compromis biais/variance 
param_grid = [
    {
        'model': [Ridge()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
    },
    {
        'model': [Lasso()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
    },
    {
        'model': [ElasticNet()],
        'model__alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
        'model__l1_ratio': [0.1, 0.5, 0.7, 0.9]
    }
]

print("Optimisation des hyperparamètres par GridSearchCV...")
gs_reg = GridSearchCV(reg_pipe, param_grid, cv=cv, scoring=scoring, refit='RMSE', n_jobs=-1)
gs_reg.fit(X_train, y_train)

best_reg_r2 = gs_reg.cv_results_['mean_test_R2'][gs_reg.best_index_]
best_reg_rmse = -gs_reg.best_score_
best_reg_mae = -gs_reg.cv_results_['mean_test_MAE'][gs_reg.best_index_]

print(f"Meilleur modèle régularisé : {type(gs_reg.best_estimator_['model']).__name__}")
print(f"Paramètres optimaux : {gs_reg.best_params_}")
print(f"  R² CV    : {best_reg_r2:.4f}")
print(f"  RMSE CV  : {best_reg_rmse:.4f}")
print(f"  MAE CV   : {best_reg_mae:.4f}")

3. MODÈLES RÉGULARISÉS (Ridge, Lasso, ElasticNet)
--------------------------------------------------
Optimisation des hyperparamètres par GridSearchCV...
Meilleur modèle régularisé : ElasticNet
Paramètres optimaux : {'model': ElasticNet(), 'model__alpha': 0.01, 'model__l1_ratio': 0.1}
  R² CV    : 0.2655
  RMSE CV  : 1.2515
  MAE CV   : 0.9236


In [169]:
print("4. MODÈLES ARBRES DECISIONNELS (GradientBoost et Random Forest)")
print("-"*50)
# modèles d'arbres décisionnels - non linéaires
# --- Pipeline “vide” pour le Gradient Boosting ---
boost_pipe = Pipeline([
    ('prep', preproc),
    ('model', GradientBoostingRegressor(random_state=42))
])

# --- Grille d’hyper-paramètres (préfixe 'model__') ---
param_grid_boost = {
    'model__n_estimators':  [100, 200, 300],
    'model__learning_rate': [0.01, 0.1, 0.2],
    'model__max_depth':     [3, 5, 7]
}

# --- Lancement de la GridSearch ---
gs_boost = GridSearchCV(
    boost_pipe,
    param_grid_boost,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    return_train_score=True,
    n_jobs=-1,
    refit='RMSE'
)
gs_boost.fit(X_train, y_train)
print("→ Meilleurs params Boosting :", gs_boost.best_params_)

# Pipeline “vide” pour Random Forest
rf_pipe = Pipeline([
    ('prep', preproc),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

# Grille de recherche (attention au préfixe model__)
param_grid_rf = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth':    [None, 5, 10]
}

# Lancement de la GridSearch
gs_rf = GridSearchCV(
    rf_pipe,
    param_grid_rf,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    refit='RMSE',
    return_train_score=True
)
gs_rf.fit(X_train, y_train)
print("→ Meilleurs params RF :", gs_rf.best_params_)


4. MODÈLES ARBRES DECISIONNELS (GradientBoost et Random Forest)
--------------------------------------------------
→ Meilleurs params Boosting : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100}
→ Meilleurs params RF : {'model__max_depth': 10, 'model__n_estimators': 300}


Méthodes d'ensemble : Bagging / Boosting / Stacking

In [170]:

print("\n5. MÉTHODES D'ENSEMBLE")
print("-" * 50)
# Techniques pour combiner plusieurs modèles
# A) BAGGING - Réduit la variance en combinant plusieurs modèles entraînés sur
# différents échantillons des données d'entraînement
print("A) BAGGING (Bootstrap Aggregating)")
bagging_pipe = Pipeline([
    ('prep', preproc),
    ('model', BaggingRegressor(
        n_estimators=50,
        random_state=42,
        n_jobs=-1
    ))
])
cv_bagging = cross_validate(bagging_pipe, X_train, y_train,cv=cv, scoring=scoring)
print(f"  R² CV   : {cv_bagging['test_R2'].mean():.4f} ± {cv_bagging['test_R2'].std():.4f}")
print(f"  RMSE CV : {-cv_bagging['test_RMSE'].mean():.4f} ± {cv_bagging['test_RMSE'].std():.4f}")
print(f"  MAE CV  : {-cv_bagging['test_MAE'].mean():.4f} ± {cv_bagging['test_MAE'].std():.4f}")

# B) BOOSTING
print("\nB) BOOSTING (Gradient Boosting)")
# Justification : Réduit le biais en combinant séquentiellement des modèles faibles,
# chaque nouveau modèle corrigeant les erreurs du précédent

boosting_pipe = Pipeline([
    ('prep', preproc),
    ('model', HistGradientBoostingRegressor(random_state=42))
])

# Optimisation des hyperparamètres pour le boosting
param_grid_boost = {
    'model__max_iter': [100, 200],
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3, 5]
}

gs_boost = GridSearchCV(boosting_pipe, param_grid_boost, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)
gs_boost.fit(X_train, y_train)

cv_boost_results = cross_validate(gs_boost.best_estimator_, X_train, y_train, cv=cv, scoring=scoring)
print(f"Paramètres optimaux : {gs_boost.best_params_}")
print(f"  R² CV    : {cv_boost_results['test_R2'].mean():.4f} ± {cv_boost_results['test_R2'].std():.4f}")
print(f"  RMSE CV  : {-cv_boost_results['test_RMSE'].mean():.4f} ± {cv_boost_results['test_RMSE'].std():.4f}")
print(f"  MAE CV   : {-cv_boost_results['test_MAE'].mean():.4f} ± {cv_boost_results['test_MAE'].std():.4f}")

# C) STACKING
print("\nC) STACKING")
# Justification : Combine les prédictions de plusieurs modèles via un meta-learner
# qui apprend comment optimiser la combinaison

# Modèles de base pour le stacking
base_models = [
    ('ridge', gs_reg.best_estimator_['model']),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)),
    ('gb', gs_boost.best_estimator_['model'])
]

stacking_pipe = Pipeline([
    ('prep', preproc),
    ('model', StackingRegressor(
        estimators=base_models,
        final_estimator=LinearRegression(),
        cv=3,  # CV interne pour éviter l'overfitting
        n_jobs=-1
    ))
])

cv_stacking = cross_validate(stacking_pipe, X_train, y_train, cv=cv, scoring=scoring)
print(f"  R² CV    : {cv_stacking['test_R2'].mean():.4f} ± {cv_stacking['test_R2'].std():.4f}")
print(f"  RMSE CV  : {-cv_stacking['test_RMSE'].mean():.4f} ± {cv_stacking['test_RMSE'].std():.4f}")
print(f"  MAE CV   : {-cv_stacking['test_MAE'].mean():.4f} ± {cv_stacking['test_MAE'].std():.4f}")



5. MÉTHODES D'ENSEMBLE
--------------------------------------------------
A) BAGGING (Bootstrap Aggregating)
  R² CV   : 0.4194 ± 0.0290
  RMSE CV : 1.1302 ± 0.1089
  MAE CV  : 0.9028 ± 0.1013

B) BOOSTING (Gradient Boosting)
Paramètres optimaux : {'model__learning_rate': 0.05, 'model__max_depth': 3, 'model__max_iter': 100}
  R² CV    : 0.3935 ± 0.0155
  RMSE CV  : 1.1543 ± 0.0965
  MAE CV   : 0.9277 ± 0.0907

C) STACKING
  R² CV    : 0.4125 ± 0.0877
  RMSE CV  : 1.1320 ± 0.1093
  MAE CV   : 0.8862 ± 0.1038


Comparaison des modèles

In [171]:
print("\n6. COMPARAISON DES MODÈLES (Validation Croisée)")
print("-" * 70)
# evaluer la meilleure performance de modèles
results_cv = {
    'Baseline (Dummy)': (
        baseline_r2,
        baseline_rmse,
        baseline_mae
    ),
    'Régression Linéaire': (
        cv_linear['test_R2'].mean(),
        -cv_linear['test_RMSE'].mean(),
        -cv_linear['test_MAE'].mean()
    ),
    'Meilleur Régularisé': (
        best_reg_r2,
        best_reg_rmse,
        best_reg_mae
    ),
    'Bagging': (
        cv_bagging['test_R2'].mean(),
        -cv_bagging['test_RMSE'].mean(),
        -cv_bagging['test_MAE'].mean()
    ),
    'Boosting': (
        cv_boost_results['test_R2'].mean(),
        -cv_boost_results['test_RMSE'].mean(),
        -cv_boost_results['test_MAE'].mean()
    ),
    'Stacking': (
        cv_stacking['test_R2'].mean(),
        -cv_stacking['test_RMSE'].mean(),
        -cv_stacking['test_MAE'].mean()
    )
}

# Affichage du tableau
print(f"{'Modèle':<20} {'R²':<8} {'RMSE':<8} {'MAE':<8}")
print("-" * 50)
for model, (r2, rmse, mae) in results_cv.items():
    print(f"{model:<20} {r2:<8.4f} {rmse:<8.4f} {mae:<8.4f}")

# Choix du meilleur modèle sur le métrique choisi : RMSE
best_model_name = min(results_cv.keys(), key=lambda x: results_cv[x][1])
print(f"\nMeilleur modèle (RMSE le plus faible) : {best_model_name}")




6. COMPARAISON DES MODÈLES (Validation Croisée)
----------------------------------------------------------------------
Modèle               R²       RMSE     MAE     
--------------------------------------------------
Baseline (Dummy)     -0.0066  1.4870   1.1643  
Régression Linéaire  0.2266   1.2752   0.9305  
Meilleur Régularisé  0.2655   1.2515   0.9236  
Bagging              0.4194   1.1302   0.9028  
Boosting             0.3935   1.1543   0.9277  
Stacking             0.4125   1.1320   0.8862  

Meilleur modèle (RMSE le plus faible) : Bagging


Evaluation finale du meilleur modèle pour le jeu de test

In [172]:
print("\n7. ÉVALUATION FINALE SUR LE JEU DE TEST")
print("-" * 50)

# Sélection du meilleur modèle pour l'évaluation finale
if best_model_name == 'Stacking':
    final_model = stacking_pipe
elif best_model_name == 'Boosting':
    final_model = gs_boost.best_estimator_
elif best_model_name == 'Meilleur Régularisé':
    final_model = gs_reg.best_estimator_
elif best_model_name == 'Bagging':
    final_model = bagging_pipe
else:
    final_model = linear_pipe
# Entraînement et prédiction sur le test
final_model.fit(X_train, y_train)
y_pred_test = final_model.predict(X_test)

# Calcul des métriques finales
r2_test = r2_score(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae_test = mean_absolute_error(y_test, y_pred_test)

print(f"Performance du modèle final ({best_model_name}) sur le jeu de test :")
print(f"  R²   : {r2_test:.4f}") #on note ici qu'avec un taux de 0.34, il y a une part importante de la variance qui n'est pas expliquée même si c'est cohérent avec notre monde
print(f"  RMSE : {rmse_test:.4f}")
print(f"  MAE  : {mae_test:.4f}")

# Comparaison avec baseline sur test
dummy_pipe.fit(X_train, y_train)
y_pred_dummy_test = dummy_pipe.predict(X_test)
rmse_dummy_test = np.sqrt(mean_squared_error(y_test, y_pred_dummy_test))

improvement = (rmse_dummy_test - rmse_test) / rmse_dummy_test * 100
print(f"\nAmélioration vs baseline : {improvement:+.1f}%")

# Vérification de cohérence CV vs Test
cv_rmse = results_cv[best_model_name][1]
coherence = abs(cv_rmse - rmse_test) / cv_rmse * 100
print(f"Cohérence CV/Test : {coherence:.1f}% de différence")

if coherence > 15:
    print(" Attention : différence importante entre CV et test")
else:
    print("Cohérence acceptable entre CV et test")


7. ÉVALUATION FINALE SUR LE JEU DE TEST
--------------------------------------------------
Performance du modèle final (Bagging) sur le jeu de test :
  R²   : 0.3432
  RMSE : 1.1101
  MAE  : 0.8353

Amélioration vs baseline : +19.3%
Cohérence CV/Test : 1.8% de différence
Cohérence acceptable entre CV et test


In [173]:
print("\n8. CONCLUSION ET JUSTIFICATIONS TECHNIQUES")
print("-" * 70)

print(" MÉTRIQUE CHOISIE : RMSE")
print("  Justification : RMSE pénalise davantage les grosses erreurs, crucial pour")
print("  la prédiction d'émissions où les erreurs importantes sont plus problématiques.")

print("\n MÉTHODES TESTÉES :")
print("  - Baseline : établit une référence minimale")
print("  - Régression linéaire : comprend les relations de base")
print("  - Modèles régularisés : évitent l'overfitting")
print("  - Bagging : réduit la variance par moyennage")
print("  - Boosting : réduit le biais par correction séquentielle")
print("  - Stacking : optimise la combinaison de modèles")

print("\n✓ VALIDATION CROISÉE : 5-fold avec shuffle")
print("  Garantit une évaluation robuste et reproductible")

print(f"\n✓ MODÈLE FINAL SÉLECTIONNÉ : {best_model_name}")
print(f"  Performance : RMSE = {rmse_test:.4f}")
print(f"  Amélioration vs baseline : {improvement:+.1f}%")
print(f" Pour aller plus loin, on pourrait tester d’autres méthodes d’ensemble comme le XGBoost ou le LightGBM")



8. CONCLUSION ET JUSTIFICATIONS TECHNIQUES
----------------------------------------------------------------------
 MÉTRIQUE CHOISIE : RMSE
  Justification : RMSE pénalise davantage les grosses erreurs, crucial pour
  la prédiction d'émissions où les erreurs importantes sont plus problématiques.

 MÉTHODES TESTÉES :
  - Baseline : établit une référence minimale
  - Régression linéaire : comprend les relations de base
  - Modèles régularisés : évitent l'overfitting
  - Bagging : réduit la variance par moyennage
  - Boosting : réduit le biais par correction séquentielle
  - Stacking : optimise la combinaison de modèles

✓ VALIDATION CROISÉE : 5-fold avec shuffle
  Garantit une évaluation robuste et reproductible

✓ MODÈLE FINAL SÉLECTIONNÉ : Bagging
  Performance : RMSE = 1.1101
  Amélioration vs baseline : +19.3%
 Pour aller plus loin, on pourrait tester d’autres méthodes d’ensemble comme le XGBoost ou le LightGBM
